In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/train.csv')
df

In [ ]:
df.describe(include='object')

In [ ]:
# make the input = "TEXT1: abatement; TEXT2: eliminating process"
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [ ]:
df.input.head()

In [ ]:
!pip install datasets
!pip install transformers
!pip install sentencepiece

In [ ]:
from datasets import Dataset,DatasetDict

ds = Dataset.from_pandas(df)
ds

In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
#using Deberta
model_nm = '../input/debertav3small'
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
def tok_func(x): return tokz(x["input"])
tok_ds = ds.map(tok_func, batched=True)

In [ ]:
row = tok_ds[0]
row['input'], row['input_ids']

In [ ]:
tokz.vocab['▁of']

In [ ]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

**Test and Validation Sets**

In [ ]:
eval_df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')
eval_df.describe()

In [ ]:
# split training to training set and validation set
dds = tok_ds.train_test_split(0.25, seed=42)
dds

In [ ]:
#eval = test set
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

In [ ]:
import numpy as np
def corr(x,y): return np.corrcoef(x,y)[0][1]

In [ ]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

**Train Model**

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
bs = 128
epochs = 4
lr = 8e-5
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
trainer.train();

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

In [ ]:
preds = np.clip(preds, 0, 1)
preds

In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds.flatten()
})

submission.to_csv('submission.csv', index=False)